In [ ]:
import cv2
import numpy as np
from PIL import Image
from keras.models import load_model
from keras.preprocessing.image import img_to_array, array_to_img
from mtcnn import MTCNN
import matplotlib.pyplot as plt
from keras_facenet import FaceNet

# Load pre-trained models
face_detector = MTCNN()
MyFaceNet = FaceNet()
modelSmile = load_model('MODEL_BARU14.h5')

# Define labels for facial expressions
labels = ['jijik', 'marah', 'normal', 'sedih','senang','takut','terkejut']

# Access camera
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gbr1 = frame

    wajah = face_detector.detect_faces(gbr1)

    if len(wajah) == 0:
        print("No faces detected in the input image.")
    else:
        for face1 in wajah:
            x1, y1, width, height = face1['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height

            face = gbr1[y1:y2, x1:x2]

            face = Image.fromarray(face)
            face = face.resize((160, 160))
            face = np.asarray(face)

            face = np.expand_dims(face, axis=0)
            signature = MyFaceNet.embeddings(face)

            y = modelSmile(signature)

            m = np.round(y)
            ekspresi = labels[m[0].argmax()]
            accuracy = y[0][m[0].argmax()] * 100

            cv2.putText(gbr1, ekspresi, (x1, y1 + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1, cv2.LINE_AA)
            cv2.putText(gbr1, f"({accuracy:.2f}%)", (x1, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1, cv2.LINE_AA)
            cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)

            print("Facial Expression detected:", ekspresi)
            print("Accuracy:", accuracy)

    cv2.imshow('Camera', gbr1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
